# ALI的Tensorflow炼成与GAN科普


**Abstract：**Deep Learning是一个很大的领域，其中GAN是Deep Learning的明星，希望大家可以通过本文来简单的了解一下GAN这个模型以及这个模型的一些运用。本文介绍关于GAN的一些知识以及GAN的思想如何转移到ALI中，以及通过Google的Deep Learning框架`TensorFlow`，通过MNIST数据来实现ALI模型。所涉及到Tensorflow的一些很简单的一些解说，包括`tf.Variable()`和`tf.placeholder()`的一些用法以及区别，同时给出一小段代码案例。虽然提到了最简单的两个Tensorflow，`tf.Variable()`和`tf.placeholder()`，但是在本文中，不深入解释更多的关于TensorFlow的运用以及神经网络是如何搭建的。最后，我们将结果组合成模型。

## 使用Tensorflow建立ALI模型

### 背景

由于这次做项目的时候用到ALI这个Model，朋友希望自己能够为ALI和GAN来写一点自己的想法，故作此文。

生成模型已经成为建复杂高维数据集模型的常用的框架，面对复杂性时如同降维二向箔打击。

---------------------

### 什么是ALI模型？

ALI(adversarially learned inference)，中文直译过来呢，就是对抗学习推理了吧？

在这里简单的给大家介绍一下ALI模型。

既然有一个Adversarial在里面，那想必就和GAN有一点关系了。而GAN(Generative Adversarial Networks)即“生成对抗网络”就是Deep Learning入门神书的作者Ian Goodfellow的神作。

GAN的idea就是同时训练两个模型，一个生成模型，一个Discriminative Model。我们把生成模型简写成G，Discriminative Model简称为D。而实现的方法，是让两个网络相互竞争。

生成模型是用来获取数据的分布情况的，而D则是用来估计来自训练数据的概率的。

生成模型的训练过程就是想办法把D的错误概率给最大化。

在任意函数G和D的空间中，存在唯一的解决方案，G恢复训练数据分布，D等于$1/2$。 

在G和D由多层感知器定义的情况下，整个系统可以用BP来进行训练。 

在Training或Generating样本的期间，**不需要任何马尔科夫链或展开的近似推理网络。**，也就是活生生的把推理给绕过去了。当然还有Autoregressive Approaches（自回归方法）放弃潜在的表征，而是直接对输入变量之间的关系进行建模。或多或少的砍掉或者放弃掉一些东西。

自回归模型可以生成相当出色的样本，但是牺牲掉了速度。同时其要求学习之前数据的抽象表达。而基于GAN的方法代表了一个很好的妥协：他们学习一个生成模型，生成比最佳VAE技术更高质量的样本，而不牺牲采样速度，并且还利用潜在代表在生成过程中。然而，GAN缺乏有效的推理机制，使得GANs无法在抽象层面推理数据。因此大牛们在研究如何优雅的将GANs其他的方法进行结合，出杂交种，其实本质意义上就是在两个短板中互相妥协罢了。

ALI是个通过Generation Network和Inference Network。 两个Model来对怼。也就是将Inference Machine（或encoder）和深度定向G Model（decoder）投入到类似GAN的对抗框架中学习。

训练一个鉴别器，以便将来自解码器的联合样本的数据和相应的潜在变量的联合样本与编码器（或近似后验）区分开，而编码器和解码器被一起训练以愚弄鉴别器。我们不仅要求鉴别器区分合成样本与实际数据，而且要求它区分数据空间和潜在变量之间的两个联合分布。生成网络将样本从随机潜在变量映射到数据空间，而推理网络将数据空间中的训练示例映射到潜在变量的空间。

我们可以针对GAN得出以下构想：

两个一个生成器，一个判别器共两个神经网络重复博弈。

![](ALI-概念图.png)

--------------------------

今天用Tensorflow实现一下，同时解释一些Tensorflow的Concept

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

在这里，我们设置好我们的参数。

In [2]:
mb_size = 32
X_dim = 784
z_dim = 64
h_dim = 128
lr = 1e-3
d_steps = 3

导入大名鼎鼎的MNIST数据

In [3]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz


### 一个简单的科普

很多人其实经常就是Tensorflow的第一步就愣住了，卧槽？？？这是什么玩意？？？

为了让大家不至于——“卧槽这是什么玩意？”

简单的给大家说一下`tf.placeholder`：

其实大家看模型的时候（包括看Keras或者TensorLayer），其实经常可以看到两个很经典东西：

```
tf.Variable()

tf.placeholder()
```

这两个有什么区别呢？

为了训练我们的example，我们首先要立个placeholder。`tf.placeholder`是用来feed我们需要训练的example的，同时`tf.placeholder`是**必须用**`feed_dict`**来fed！！！**

给大家举个例子：

In [4]:
x_example= tf.placeholder(tf.float32, shape=(1024, 1024))
y_example = tf.matmul(x_example, x_example)

In [5]:
with tf.Session() as sess:
#     print(sess.run(y_example))
    rand_array = np.random.rand(1024, 1024)
    print(sess.run(y_example, feed_dict={x_example: rand_array}))

[[ 262.44055176  264.40164185  261.80392456 ...,  264.89172363
   265.04611206  262.62728882]
 [ 262.56356812  261.39038086  256.91809082 ...,  265.66696167
   269.59469604  269.17672729]
 [ 257.39651489  258.3777771   257.86535645 ...,  261.78411865
   254.41978455  256.24316406]
 ..., 
 [ 256.59402466  260.95648193  259.51330566 ...,  262.58615112
   265.44903564  265.43508911]
 [ 250.66888428  251.14041138  255.3341217  ...,  255.5091095   256.3104248
   258.73748779]
 [ 253.89770508  251.85395813  257.89154053 ...,  251.33482361
   254.21363831  258.79104614]]


在这里呢，`sess.run(y_example)`是会被弄死的，因为你只能用feed_dict来fed我们的`tf.placeholder`

但是我们对比一下`tf.Variable`的话，`tf.Variable`是通过`run()`调用来维护状态的，也就是说，在之前的我们的`sess.run(y_example)`是可以受用于`tf.Variable`的。

通过构造`tf.Variable`来向图中添加变量。

同时`tf.Variable()`的构造函数需要给定任何类型和形状的Tensor变量的初始值。

初始值定义变量的类型和形状，其变量的`type`和`shape`是固定的。

因此，在这里我们主要使用`tf.placeholder`来feed我们需要训练的example的。

OK，现在进入正题

In [6]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

先定义几个简单的小函数，包括一些必要的log和sample的函数以及后期用到的导出结果的可视化函数。

由于这一环节没有任何的难点，不做多余阐述：

In [7]:
def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def log(x):
    return tf.log(x + 1e-8)

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [8]:
D_W1 = tf.Variable(xavier_init([X_dim + z_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

In [9]:
Q_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
Q_W2 = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2 = tf.Variable(tf.zeros(shape=[z_dim]))

In [10]:
P_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
P_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
P_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

In [11]:
theta_G = [Q_W1, Q_W2, Q_b1, Q_b2, P_W1, P_W2, P_b1, P_b2]
theta_D = [D_W1, D_W2, D_b1, D_b2]

考虑x和z这两个概率分布：

* encoder（编码器）联合分布：$q(x,z)=q(x)q(z|x)$
* decoder（解码器）联合分布：$p(x,z)=p(z)p(x|z)$

这两种分布的边际我们是已知的，编码器边际$q(x)$是经验数据分布，解码器边缘$p(z)$通常被定义为简单的因式分布。比如标准正态分布$p(z)=N(0,I)$。p和q之间的生成过程相反。

ALI的目标是匹配两个联合分布。首先我们要确保所有边界匹配，所有条件分布也可以匹配成功。也就是我们要注意条件$q(z|x)$与$p(z|x)$的匹配。

$$\min\limits_{G}\max\limits_{D}V(D,G)=\mathbb{E}_{q(x)}[\log(D(x,G_z(x)))]+\mathbb{E}_{p(z)}[\log(1-D(G_x(z),z))]=\\ \int\int q(x)q(z|x)\log(D(x,z))dxdz\\+\int\int p(z)p(x|z)\log(1-D(x,z))dxdz$$

我们现在来定义$q(x)$和$p(z)$:

In [12]:
def Q(X):
    h = tf.nn.relu(tf.matmul(X, Q_W1) + Q_b1)
    h = tf.matmul(h, Q_W2) + Q_b2
    return h

def P(z):
    h = tf.nn.relu(tf.matmul(z, P_W1) + P_b1)
    h = tf.matmul(h, P_W2) + P_b2
    return tf.nn.sigmoid(h)

def D(X, z):
    inputs = tf.concat([X, z], axis=1)
    h = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    return tf.nn.sigmoid(tf.matmul(h, D_W2) + D_b2)

In [13]:
z_hat = Q(X)
X_hat = P(z)

In [14]:
D_enc = D(X, z_hat)
D_gen = D(X_hat, z)

定义loss

In [15]:
D_loss = -tf.reduce_mean(log(D_enc) + log(1 - D_gen))
G_loss = -tf.reduce_mean(log(D_gen) + log(1 - D_enc))

In [16]:
D_solver = (tf.train.AdamOptimizer(learning_rate=lr)
            .minimize(D_loss, var_list=theta_D))
G_solver = (tf.train.AdamOptimizer(learning_rate=lr)
            .minimize(G_loss, var_list=theta_G))

现在我们通过`tf.Session()`来跑模型。

最后，说一下这个很经常出现`Session()`。

Tensorflow以Graph的计算而得名，而`Session()`封装了执行Operation对象的环境。通过Session可以执行图的计算。

与`Session`相近的有`tf.InteractiveSession()`

`tf.InteractiveSession()`将自己安装为构建中的默认Session。

而简单直接的方法也有这般：

```python
sess = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
# We can just use 'c.eval()' without passing 'sess'
print(c.eval())
sess.close()
```

常规会话会在with语句中创建时将其自身作为默认会话。一般我们在代码中的用法是这样的，我也比较常用`Session()`这个方法就是了：

```python
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
with tf.Session():
    # We can also use 'c.eval()' here.
    print(c.eval())
```

总结一下一般的启动逻辑：

1. 先定义好变量(Variable或者placeholder)
* sess=tf.Session()做一个准备
* sess.run()，在run里面进行各种花式的张量计算，这也就是体现技术的point
* sess.close()

因此，`tensorflow`就将底层给配置好之后，我们在`tensorflow`这个大环境里愉快的奔跑。

而难点就在于Variable的各种的设置和run的操作方法。

初期，Variable被大量的套路给设定好，我们可以通过别人的轮子，来配适。

但是Deep Learning的从业者的核心价值应该在于**能够自己的将自己想做的数据导入自己的模型**，而不是简单的调参或者利用别人的轮子滚来滚去。各种算法和一些主流的实现就像加减乘除一般了然于胸，并且有自己做轮子的能力。

In [17]:
sess = tf.Session()

In [18]:
sess.run(tf.global_variables_initializer())

将模型训练十万次（如果有想法的，请换一台支持GPU的电脑，要不然可能会等很久。），我们将1000Iteration为一个单位，print一次：

In [20]:
i = 0

for it in range(100000):
    X_mb, _ = mnist.train.next_batch(mb_size)
    z_mb = sample_z(mb_size, z_dim)

    _, D_loss_curr = sess.run(
        [D_solver, D_loss], feed_dict={X: X_mb, z: z_mb}
    )

    _, G_loss_curr = sess.run(
        [G_solver, G_loss], feed_dict={X: X_mb, z: z_mb}
    )

    if it % 1000 == 0:
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'
              .format(it, D_loss_curr, G_loss_curr))

        samples = sess.run(X_hat, feed_dict={z: sample_z(16, z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'
                    .format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

Iter: 0; D_loss: 0.003775; G_loss: 26.54
Iter: 1000; D_loss: 0.06725; G_loss: 27.79
Iter: 2000; D_loss: 0.1544; G_loss: 18.63
Iter: 3000; D_loss: 0.09687; G_loss: 17.99
Iter: 4000; D_loss: 0.559; G_loss: 14.4
Iter: 5000; D_loss: 0.2624; G_loss: 13.34
Iter: 6000; D_loss: 0.4579; G_loss: 13.86
Iter: 7000; D_loss: 0.5003; G_loss: 14.18
Iter: 8000; D_loss: 0.4746; G_loss: 10.15
Iter: 9000; D_loss: 0.7304; G_loss: 10.96
Iter: 10000; D_loss: 0.6929; G_loss: 8.373
Iter: 11000; D_loss: 0.4615; G_loss: 9.82
Iter: 12000; D_loss: 0.5451; G_loss: 7.258
Iter: 13000; D_loss: 0.4942; G_loss: 7.463
Iter: 14000; D_loss: 0.7117; G_loss: 8.421
Iter: 15000; D_loss: 0.3711; G_loss: 7.135
Iter: 16000; D_loss: 1.271; G_loss: 9.167
Iter: 17000; D_loss: 0.379; G_loss: 8.942
Iter: 18000; D_loss: 0.444; G_loss: 8.204
Iter: 19000; D_loss: 0.573; G_loss: 7.7
Iter: 20000; D_loss: 0.4686; G_loss: 7.426
Iter: 21000; D_loss: 0.3614; G_loss: 8.388
Iter: 22000; D_loss: 0.5678; G_loss: 7.96
Iter: 23000; D_loss: 0.38; G_l

由于导出大量的图片，我在这里就给大家看一下对比图：

最早的图：

![](out\000.png)

不久之后：
![](out\051.png)

-----------------

本文由[ALI Paper, arxiv](https://arxiv.org/abs/1606.00704)依照Tensorflow重现。ALI的概念非本人提出，本人根据自己对ALI的理解通过Tensorflow进行实现，并且运行于Jupyter中。所有代码在Windows10,Python3.5,TensorflowGPU版(1.1.0rc2)完美运行。限于本人水平，可能有出现一定的错误，如有失误，欢迎交流。同时，由于本人学习Deep Learning的时候直接接触英文材料，故一些专有名词的翻译可能存在一些偏差。故一些专有名词直接保留为原单词，不做翻译。ALI的翻译没有参考过任何相关中文信息，由本人直接查阅在Arxiv上ALI的原文所得。

如果有想来交流的小伙伴，欢迎私信。

-----------------
## Reference

1. [ALI Paper, arxiv](https://arxiv.org/abs/1606.00704)
* [GAN - Paper, NIPS](http://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf)
* [GAN - Goodfellow](https://github.com/goodfeli/adversarial)
* [生成模型与判别模型 zouxy09,csdn](http://blog.csdn.net/zouxy09/article/details/8195017)
* [Adversarial machine learning - ACM Digital Library](https://dl.acm.org/citation.cfm?id=2046692)
* [Running Graph](https://www.tensorflow.org/api_guides/python/client)